In [1]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 12.5 MB/s eta 0:00:00


In [2]:
import enum
import hashlib
import re
from typing import Any
from pydantic import (BaseModel,EmailStr,Field,field_validator,model_validator,SecretStr,ValidationError,)

# Funções Regex (Nome e Senha)


Diferente da validação simples do exemplo 1, neste segundo caso as funções regex são mais avançadas, pois permitem personalizar com mais precisão a estrutura do campo a ser preenchido.

In [4]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

# Papel do Usuário

In [7]:
class Role(enum.IntFlag): # definindo classe
    Author = 1 # gerando inumeraçaõ explicita com numeros em potencias de 2
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# Classe User e Validações Adicionais

In [10]:
class User(BaseModel): # Definindo classe user com BaseModel do pydantic
    name: str = Field(examples=["Arjan"]) # nome
    email: EmailStr = Field( # email
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field( # senha
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8] # opçoes disponíveis de papéis de usuário
    )

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str: # essa função define atravez de expressões regulares que nome deve conter apenas letras e ter no minimo duas
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before") # validação no campo role deve ser feita antes da conversão automatica de tipo feita pelo pydantic
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), # converte int para role,
              str: lambda x: Role[x], # converte str para role
              Role: lambda x: x}      # se já estiver como role, retorna sem alterções
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]: # validação de campos de usuário
        if "name" not in v or "password" not in v:            # garantir que sejam preenchidos nome e senha
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():  # garantir que nome não esteja em senha
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):     # garantir que senha tenha a estrutura definida pela experessão regular
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest() # senha seja convertida atravez de uma função hash
        return v

# Função de Validação

In [11]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

# Função de teste

In [13]:
def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

# Execução de testes

1. campos preenchidos corretamente
2. erro: papel programes fora das opçoes disponíveis
3. erro: senha inválida, pela estrutura incompleta
4. erro: nome incorreto por conter caracteres diferentes de strings
5. erro: nome não pode estar contido em senha
6. erro: ausencia de campos obrigatórios



In [15]:
if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://